# Getting Started with the Interactive Brokers API

Here at Robot Wealth, we trade with Interactive Brokers (IB) primarily because they offer access to global markets at a reasonable price.  

In recent times, IB has put some time and effort into upping its tech game, including development of an API for interacting with its desktop trading applications. 

An application that interacts with IB's desktop trading applications via the API is essentially a message-handling program. So I want to show you a simple but effective architecture for managing the flow of messages and the operations they trigger. If you can understand this framework, then you can basically design any application you can think of. 

Before we get to that, I'll show you how to get started with the IB API - how it works, how to configure it, and options available for using it. 

Then I'll show you a Hello World example, and in later articles, I'll go through some more interesting and complex use cases.

## The IB API is (kind of) different

The IB API is essentially a message protocol for communicating with the IB desktop trading platforms - Trader Workstation (TWS) and Gateway (GW). 

If you think about this in terms of the normal client-server framework, your trading code is essentially the "client" and TWS is the "server", and they communicate using the IB API. TWS acts as an intermediary to IB's actual servers, and the API provides an interface to TWS. 

The most obvious implication is that you need to have TWS or GW running in order to use the API. 

This means that you also need to deal with restarts. 

### Dealing with restarts

TWS and GW were designed to be restarted daily (for example to re-download contract definitions where contracts have been changed or new contracts added). However, they have an auto-restart feature that restarts the application daily without user intervention. With this option enabled, the application can run for up to a week without re-authenticating. However, after the nightly server reset on Saturday night (US), you will have to re-authenticate.

Of course, if you're trading, you'll be checking on the platform constantly, but it's nice not to have to re-authenticate every time it restarts. 

To set that option in TWS, from the File menu, choose Global Configuration:

![]()

Then, from the Configuration menu, choose "Lock and Exit". Check the "Auto restart" button, uncheck the "Auto log off" button, and set the time you'd like to have the application restart:

![]()

The restart time is in the time zone you have set for TWS. You can see TWS' current time in the top right hand corner:

![]()

If you need to change the time zone, you'll need to log out, and from the log in screen choose "More options". You can then change your time zone:

![]()

## Using the IB API

You have a few options for working with the IB API. You can either use an existing wrapper such as [ib_insync](https://github.com/erdewit/ib_insync), or work with the native API. 

The advantage of something like ib_insync is that all of the "plumbing" code is done for you - in particular, it's a fully asynchronous framework which handles messages to and from TWS. 

The native API is a bit more complex to work with, and you'll have to figure out how to handle messages yourself (async, threading, etc). On the other hand, the native API is officially supported by IB (ib_insync is third-party software, although at the moment is actively maintained), and it removes any limitations on customisation. 

In this series, we'll use the native API. 

## Installing the native IB API

The first step is installing the native API. 

Grab the software installer for your operating system from [IB's github pages](https://interactivebrokers.github.io/). Run the installer and follow the prompts. 

You'll also need Python - [Miniconda](https://docs.anaconda.com/free/miniconda/index.html) provides a good, minimal installation, but use whatever you normally work with. 

Once the API software is installed, navigate to the installation directory (on Windows, usually C:\TWS API). Then drill down to the pythonclient directory and run the setup.py file with the `install` parameter:

![]()

Once the installation completes, verify that it worked by doing `python -m pip show ibapi` to show the latest installed version:

![]()

## Configuring TWS to use the API

Next we need to configure TWS. 

### Enable logging

TWS can be configured to write a separate log file that documents communication via the API. This will make debugging a whole lot simpler, so enable it in TWS as follows:

From the File menu, select "Global Configuration". Then, select "API" --> "Settings" and check the box for "Create API message file":

![]()

TWS logs are encrypted. To read them, you'll need to export them from TWS as follows:

From the Help menu, select "Troubleshooting" --> "Diagnostics" --> "API Log Files":

![]()

You can then select the logs you want to export. 

See [here](https://ibkrcampus.com/ibkr-api-page/trader-workstation-api/#tws-logs) for more detailed information about the TWS log files.

### Configuration settings

You'll also need to configure the following settings in TWS:

From the File menu, select "Global Configuration" --> "API" --> "Settings". Then, make the following changes:

- Check the box for "Enable ActiveX and Socket Clients"
- Designate a socket port - 7496 is the default
- Check the box for "Allow connections from localhost only" - this will allow connections only from applications running on the same machine as TWS.

![]()

If you want to have multiple TWS instances running on the same machine, you can configure each with a different API socket port number.

You may wish to connect to TWS remotely. In that case, uncheck "Allow connections from localhost only" and add the relevant IP addresses to "Trusted IPs". If you don't add these IPs, you'll have to manually authenticate each connection from a pop-up box in TWS (which kind of defeats the purpose of remote access).

## Understanding the messaging framework

A Python client application using the IB API requires two threads of execution:
- One for messages sent from TWS to the client application. This uses the `IBAPI.Ewrapper` interface, which you can use out of the box or overwrite depending on how you want to handle these messages. 
- One for messages sent from the client application to TWS. This uses the `IBAPI.EClientSocket` class, which you inherit from, and include your `IBApi.EWrapper` in the constructor parameters (so that the application can handle all the messages coming from TWS).

The base class `Ewrapper` defines and implements functions for handling messages from TWS by simply logging them. You may wish to inherit from this class and override the methods depending on how you want to handle these messages. 

In the Python implementation of `EClient`, messages are processed using a `Queue`. This happens in an endless loop in `Eclient.run`. In other languages, you need a third thread of execution to handle this. 

Let's make this a bit more real with a Hello World example. 

## Hello World

In this example, we'll connect to TWS, request our account information, and print it to screen. 

Before writing any code, it's always worth thinking about the architecture of an IB API application, which is inherently a message-handling program. 

Remember that we handle our messages to and from TWS in separate threads, so ideally we would have some way of signalling between threads when a message has been received or processed. 

To do this, we'll use `threading.Event`, which is a simple way to control the flow of a program hat involves asynchronous operations and multiple threads.

`threading.Event` manages an internal flag that can be set to True with the `set()` method and reset to False with the `clear()` method. Other threads can wait for this flag to be set by calling the `wait()` method, which blocks until the flag becomes True. 

This mechanism is particularly useful for signaling between threads, allowing one thread to signal another that an event has occurred or a condition has been met - which is exactly what we need to do with our TWS messaging application. 

We'll use the `Eclient.reqAccountSummary` method to request our account information. If you look at the [documentation](https://ibkrcampus.com/ibkr-api-page/trader-workstation-api/#requesting-account-summary) for this function, you'll also find the `Ewrapper` methods that we need to override in order to handle the messages coming back from TWS. 

We also use the following methods from `Eclient`:
- `connect` - initiates a connection with TWS and triggers the `Ewrapper.nextValidId` callback once established. We'll use a `threading.Event` to signal when this callback is triggered and our connection is established (messages sent while the connection is being established can be lost).
- `disconnect` - terminates the connection with TWS (but doesn't cancel in-flight orders), good practice to include this at the completion of your script. 
- `run` - processes the message queue in an infinite loop while connected.

Here's the script for connecting to TWS, requesting our account information, and printing it to screen. I've heavily commented it so that you can follow what's going on if you're new to this. 

Here's the output of running this script:

## Conclusion

Reiterate that it's a message-handling program. 
Can achieve a lot with the use of `threading.Event` to signal between threads when a message has been received and/or processed. 
In future articles, we'll explore some more complex use cases. 

You might consider having a look through the IB API documentation - link. 
Also have a look at the source code for Eclient and Ewrapper. They live in the directory...


IBApi.EClientSocket.eConnect function connects
502 error indicates connection failed - is API configured properly in TWS?

After the highest version number which can be used for communication is established, TWS will return certain pieces of data that correspond specifically to the logged-in TWS user’s session. This includes (1) the account number(s) accessible in this TWS session, (2) the next valid order identifier (ID), and (3) the time of connection. In the most common mode of operation the EClient.AsyncEConnect field is set to false and the initial handshake is taken to completion immediately after the socket connection is established. TWS will then immediately provides the API client with this information.

Important: The IBApi.EWrapper.nextValidID callback is commonly used to indicate that the connection is completed and other messages can be sent from the API client to TWS. There is the possibility that function calls made prior to this time could be dropped by TWS.


## Hello world
Connect
Get account info
Get current positions inc. cash and buying power
Buy TSLA stock MOC


-  /TWS API/source/pythonclient. In this folder, run the python3 setup.py install file to install the API as a package.
    - Then can do import ibapi
- Configuring TWS
- Examples of use probably best way to go:
    - Basics
        - Install and connect
        - Collect data?
        - Needs an open connection - ib-insync uses asyncio, can also use threading
        - Eclient and Ewrapper
        - Retrieving market data (Current, historical)
        - Firing an order
    - Rebalancing a portfolio to equal dollar weight
    - Rebalancing a portfolio to equal volatility weight
    - Pair trading
    - Market scanner
    - Calendar spread
    - Buying and selling options
        - Putting together a straddle


Next steps:
